### 텍스트 정규화

In [18]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [19]:
type(news_data)

sklearn.utils._bunch.Bunch

In [20]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [21]:
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index()) # 20개의 인덱스
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


# 분류 실습을 위한 데이터 선정

*  subset='train'으로 학습용(Train) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
*  기사내용으로만 분류 예측하기 위해 'headers', 'footers', 'quotes' 제거
    * Headers
        * 뉴스그룹의 메시지 헤더, 예를 들어 작성자, 이메일 주소, 뉴스그룹의 이름 등이 제거
        * 이 정보는 종종 문서의 실제 내용과 무관하게 분류 작업에 영향을 미칠 수 있기 때문에 제거하도록 시도.
    * Footers
        * 기사의 바닥글 부분, 예를 들어 작성자의 서명 라인이나 연락처 정보가 제거됨
        * 이 역시 문서의 주제 분류에 영향을 줄 수 있는 정보이기 때문에 제거합니다.
    * Quotes
        * 기사 내에서 다른 소스나 이전 게시물에서 인용된 텍스트 부분이 제거됨.
        * 이는 문서의 원본 내용을 명확히 하고, 인용문에 의존하지 않고 원본 내용만으로 분류할 수 있게 합니다.


* 전체 항목을 포함한 데이터 셈플

In [22]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* Headers 내용

* Footers 내용

* Quotes
    * qhxhd '>' 기호로 시작

In [23]:
from sklearn.datasets import fetch_20newsgroups
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

<class 'list'>


In [24]:
X_train[0]

"\n\nWhat I did NOT get with my drive (CD300i) is the System Install CD you\nlisted as #1.  Any ideas about how I can get one?  I bought my IIvx 8/120\nfrom Direct Express in Chicago (no complaints at all -- good price & good\nservice).\n\nBTW, I've heard that the System Install CD can be used to boot the mac;\nhowever, my drive will NOT accept a CD caddy is the machine is off.  How can\nyou boot with it then?\n\n--Dave\n"

In [25]:
y_train[0]

4

In [26]:
# subset='test'으로 테스트(Test) 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

학습 데이터 크기 11314 , 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행. 
# CountVectorizer: 단어의 빈도수로 Featuring
cnt_vect = CountVectorizer()

# 훈련데이터 자연어 변환
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
X_train_cnt_vect

<11314x101631 sparse matrix of type '<class 'numpy.int64'>'
	with 1103627 stored elements in Compressed Sparse Row format>

In [28]:
# 시험데이터 자연어 변환
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


### 테스트1 - 단어빈도 + Logistic Regression 기본 예측

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.607


### 테스트2 - TF-IDF + Logistic Regression 기본 예측

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
# 기본 use_idf 매개변수의 기본값은 True
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


TF-IDF Logistic Regression 의 예측 정확도는 0.674


### 테스트3 - TF-IDF + 불용어 처리 + 바이그램 처리 + LogisticRegression 기본 예측

* 유니그램 (TfidfVectorizer 기본 옵션)
    * "아버지 가방에" 인경우 아래와 같이 처리
        * 아버지, 가방에
* 바이그램: 단어와 단어 사이의 조합을 고려하여 2개의 복합단어까지 처리
    * "아버지 가방에" 인경우 아래와 같이 처리
        * 아버지, 가방에, 아버지가방에
    * 장점: 복합명사등에 대한 의미 처리가 가능하기 때문에 예측력이 올라갈수 있다.
          * 인재 개발 => 인재, 개발, 인재개발
          * 여우 비 => 여우, 비, 여우비
    * 단점: 조합가능한 모든 단어에 대한 조합을 하기 때문에 전처리 시간과 분석시간이 기하급수적으로 늘어날수 있다.
* 그 이상은 트라이그램, N 그램으로 확장하나 리소스 문제와 다중공선성(차원의저주)문제를 고려해야 한다.

In [31]:
# stop_words='english' : 영어 단어중 불용어를 제거하고 분석을 한다.
# stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경하여 Feature Vectorization 적용.
# max_df: 자주 사용되는 단어 제한
# tfidf_vect = TfidfVectorizer(stop_words='english')
# tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 Fold셋으로 설정. 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
# verbose=0(default)면 메시지 출력 안함
# verbose=1이면 간단한 메시지 출력
# verbose=2이면 하이퍼 파라미터별 메시지 출력

grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params , cv=3 , scoring='accuracy' , verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 수행하고 정확도 평가. 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))


Fitting 3 folds for each of 5 candidates, totalling 15 fits


# 과제
lightGBM을 활용하여 모델 성능을 향상 시켜보세요